In [20]:
import json
import os
import glob


In [21]:
"""Read log.txt file, which stores the quotient 
between the chemical formula of the unit cell and the chemical formula from CSD
"""
ref_codes = []
multipliers = []

with open('log.txt', 'r') as f:
    for line in f:
        data = line.strip().split(',')
        if data[1] == 'FAIL':
            continue
        else:
            if int(data[1]) > 0:
                ref_codes.append(data[0])
                multipliers.append(int(data[1]))
            else:
                print(data[0])


In [22]:
with open('data/Anionic_MOFs_formula.json', 'r') as f:
    chemical_formulas_data = json.load(f)


In [23]:
basepath = 'example_cifs/after_solvent_removal'

for idx, ref in enumerate(ref_codes):
    net_charge_per_form = chemical_formulas_data[ref]['framework'][0]['net_charge_per_formula']
    net_charge_per_cif = net_charge_per_form*multipliers[idx]
    cifpath = os.path.join(basepath, ref + '.Non-disordered_MOF_subset_removed_ions.cif')
    atoms = ase_read(cifpath)
    mass = sum(atoms.get_masses())
    volume = atoms.get_volume()
    nMg = net_charge_per_cif/2
    nH2 = 3*nMg
    H2_per_mass = 2* nH2 / mass
    H2_per_volume = 2* nH2/volume* 1660

    with open('estimated_H2_capacity.csv', 'a') as f:
        f.write('{},{},{}\n'.format(ref, H2_per_mass, H2_per_volume))
